In [1]:
import matplotlib.pyplot as plt
import spcancestry
from gnomad.sample_qc.ancestry import assign_population_pcs

import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import label_binarize

# 1. Comparing SPCAncestry stacking with gnomAD RF

## 1A. HGDP1KG data

### SPCAncestry stacking

In [3]:
path = '/Volumes/ExternalDrive/SPCAncestry/data/hgdp_1kg/'
inref_mt = spcancestry.Read(file=f'{path}hgdp_1kg_truth.bed', qc=False).as_matrixtable()
input_mt = spcancestry.Read(file=f'{path}hgdp_1kg_unknown.fam', qc=False).as_matrixtable()

2023-05-09 00:31:03.916 Hail: INFO: Found 3021 samples in fam file.
2023-05-09 00:31:03.916 Hail: INFO: Found 199974 variants in bim file.
2023-05-09 00:31:04.833 Hail: INFO: Found 335 samples in fam file.
2023-05-09 00:31:04.833 Hail: INFO: Found 199974 variants in bim file.


In [4]:
# The following will:
# 1. Intersect the two datasets to make sure we are using the same set of SNPs for training+inference
# 2. Compute PCs using reference data (from the 1)
# 3. Project unknown samples onto reference PC space (from 2)
scores_df, colnames = spcancestry.PCProject(ref_mt=inref_mt, data_mt=input_mt,
                                            ref_info=f'{path}hgdp_1kg_truth_labels.txt').run_pca_projection()

Intersecting


(199974, 3021)
(199974, 335)
Running reference PCA


2023-05-09 00:31:51.583 Hail: INFO: hwe_normalize: found 199974 variants after filtering out monomorphic sites.
2023-05-09 00:32:07.037 Hail: INFO: pca: running PCA with 20 components... / 10]
2023-05-09 00:49:34.701 Hail: INFO: Coerced sorted dataset         (8 + 2) / 10]
2023-05-09 00:50:11.504 Hail: INFO: wrote table with 199974 rows in 10 partitions to /tmp/persist_TableDM34xXtziq
2023-05-09 00:50:13.208 Hail: INFO: Reading table without type imputation
  Loading field 'Sample' as type str (not specified)
  Loading field 'SuperPop' as type str (not specified)
2023-05-09 00:50:13.860 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-05-09 00:50:14.137 Hail: INFO: Coerced sorted dataset
2023-05-09 00:50:16.616 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2023-05-09 00:50:53.853 Hail: INFO: Coerced sorted dataset          (1 + 1) / 2]


In [5]:
# The output will contain PCs for both the training and unknown samples
# Unknown samples will have NA in the SuperPop column
scores_df

,s,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,SuperPop
0,HG00096,-0.130653,0.250466,-0.027431,-0.082609,-0.014573,0.001607,0.06208,0.018693,-0.000118,...,-0.00559,-0.000164,-0.009424,0.005099,-0.002693,0.016227,-0.01332,0.003618,0.000119,EUR
1,HG00097,-0.129984,0.247212,-0.024903,-0.084724,-0.011458,0.002756,0.060606,0.017586,-0.003013,...,-0.006396,-0.003164,-0.009694,0.00103,-0.001513,0.007614,-0.002749,0.004455,-0.00541,EUR
2,HG00099,-0.130497,0.246637,-0.026746,-0.083507,-0.008789,0.002276,0.064794,0.015948,-0.00385,...,-0.004857,0.001914,-0.008211,0.001449,-0.002011,0.01137,0.000295,0.003164,0.001668,EUR
3,HG00100,-0.12885,0.250958,-0.024698,-0.088106,-0.011255,0.00257,0.050964,0.018685,-0.003741,...,-0.016691,0.000592,-0.006142,0.003748,0.001364,0.010581,-0.000015,0.002789,-0.000287,EUR
4,HG00103,-0.129443,0.246409,-0.025154,-0.089888,-0.009347,0.005414,0.059072,0.017929,-0.001648,...,-0.014377,-0.002957,-0.007067,0.000623,0.00004,0.008775,-0.001966,-0.003394,-0.000298,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,NA20903,-0.123742,0.083491,0.125892,0.132143,0.025654,-0.000135,0.008899,0.004622,-0.00147,...,0.003159,-0.00067,-0.00364,-0.001181,0.002269,0.006341,0.001097,-0.002091,0.001862,<NA>
331,NA21108,-0.124049,0.109607,0.099954,0.106831,0.019539,-0.001934,0.008966,-0.004078,-0.001156,...,-0.001278,-0.002778,0.002377,0.000648,-0.003439,0.006679,-0.002601,0.002786,0.008124,<NA>
332,NA21114,-0.125164,0.066609,0.13435,0.151027,0.029075,-0.001849,0.011448,0.007847,0.002058,...,0.003778,-0.000728,-0.00489,-0.004165,0.001264,-0.002094,0.001562,-0.000529,-0.009356,<NA>
333,NA21129,-0.129031,0.063162,0.141917,0.15565,0.029053,-0.001486,0.011335,0.002948,0.001191,...,0.002679,-0.001316,-0.004069,0.001601,-0.002589,-0.001069,0.000738,0.002867,-0.012532,<NA>


In [6]:
# infer ancestry using spcancestry and gnomAD RF
spcancestry_infered = spcancestry.infer_ancestry(scores_df, colnames)

Estimated error rate for the meta model is 0.0016528925619834212


/usr/local/lib/python3.9/site-packages/spcancestry/spcancestry.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_data.loc[:, output_col] = clf.predict(unknown_data[pc_cols].values)


### gnomAD RF

In [7]:
import logging
import random
from typing import Any, Counter, List, Optional, Tuple, Union

import hail as hl
import pandas as pd

logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def assign_population_pcs_bug_fixed(
    pop_pca_scores: Union[hl.Table, pd.DataFrame],
    pc_cols: Union[hl.expr.ArrayExpression, List[int], List[str]],
    known_col: str = "known_pop",
    fit: Any = None,  # Type should be RandomForestClassifier but we do not want to import sklearn.RandomForestClassifier outside
    seed: int = 42,
    prop_train: float = 0.8,
    n_estimators: int = 100,
    min_prob: float = 0.9,
    output_col: str = "pop",
    missing_label: str = "oth",
    pc_expr: Union[hl.expr.ArrayExpression, str] = "scores",
) -> Tuple[
    Union[hl.Table, pd.DataFrame], Any
]:  # 2nd element of the tuple should be RandomForestClassifier but we do not want to import sklearn.RandomForestClassifier outside
    """
    Use a random forest model to assign population labels based on the results of PCA.
    Default values for model and assignment parameters are those used in gnomAD.
    As input, this function can either take:
        - A Hail Table (typically the output of `hwe_normalized_pca`). In this case,
            - `pc_cols` should be one of::
                - A list of integers where each element is one of the PCs to use.
                - A list of strings where each element is one of the PCs to use.
                - An ArrayExpression of Floats where each element is one of the PCs.
                  to use
            - A Hail Table will be returned as output.
        - A Pandas DataFrame. In this case:
            - Each PC should be in a separate column and `pc_cols` is the list of all
              the columns containing the PCs to use.
            - A pandas DataFrame is returned as output.
    .. note::
        If you have a Pandas Dataframe and have all PCs as an array in a single column,
        the `expand_pd_array_col`can be used to expand this column into multiple `PC`
        columns.
    :param pop_pca_scores: Input Hail Table or Pandas Dataframe.
    :param pc_cols: List of which PCs to use/columns storing the PCs to use. Values
        provided should be 1-based and should be a list of integers when passing in a
        Hail Table (i.e. [1, 2, 4, 5]) or a list of strings when passing in a Pandas
        Dataframe (i.e. ["PC1", "PC2", "PC4", "PC5"]). When passing a HT this can also
        be an ArrayExpression containing all the PCs to use.
    :param known_col: Column storing the known population labels.
    :param fit: Fit from a previously trained random forest model (i.e., the output
        from a previous RandomForestClassifier() call).
    :param seed: Random seed.
    :param prop_train: Proportion of known data used for training.
    :param n_estimators: Number of trees to use in the RF model.
    :param min_prob: Minimum probability of belonging to a given population for the
        population to be set (otherwise set to `None`).
    :param output_col: Output column storing the assigned population.
    :param missing_label: Label for samples for which the assignment probability is
        smaller than `min_prob`.
    :param pc_expr: Column storing the list of PCs. Only used if `pc_cols` is a List of
        integers. Default is scores.
    :return: Hail Table or Pandas Dataframe (depending on input) containing sample IDs
        and imputed population labels, trained random forest model.
    """
    from sklearn.ensemble import RandomForestClassifier

    hail_input = isinstance(pop_pca_scores, hl.Table)
    if hail_input:
        if isinstance(pc_cols, list):
            if not all(isinstance(n, int) for n in pc_cols):
                raise TypeError(
                    "Using a Hail Table with a list of PC cols to use (pc_cols) "
                    "requires all values of the pc_cols list to be integers."
                )
            if isinstance(pc_expr, str):
                pc_expr = pop_pca_scores[pc_expr]
            pcs_to_pull = [pc_expr[i - 1] for i in pc_cols]
        else:
            pc_col_len = list(
                filter(
                    None,
                    pop_pca_scores.aggregate(hl.agg.collect_as_set(hl.len(pc_cols))),
                )
            )
            if len(pc_col_len) > 1:
                raise ValueError(
                    "More than one length was found among the 'pc_cols' "
                    "ArrayExpression values. The length must be consistent!"
                )
            pcs_to_pull = pc_cols
            pc_cols = list(range(1, pc_col_len[0] + 1))
        if not fit:
            pop_pca_scores = pop_pca_scores.select(known_col, pca_scores=pcs_to_pull)
        else:
            pop_pca_scores = pop_pca_scores.select(pca_scores=pcs_to_pull)

        pop_pc_pd = pop_pca_scores.to_pandas()

        # Explode the PC array
        pc_cols = [f"PC{i}" for i in pc_cols]
        pop_pc_pd[pc_cols] = pd.DataFrame(pop_pc_pd["pca_scores"].values.tolist())

    else:
        if not all(isinstance(n, str) for n in pc_cols):
            raise TypeError(
                "Using a Pandas DataFrame with pc_cols requires all values of the"
                " pc_cols list to be strings."
            )
        pop_pc_pd = pop_pca_scores

    # Split training data into subsamples for fitting and evaluating
    if not fit:
        train_data = pop_pc_pd.loc[~pop_pc_pd[known_col].isnull()]
        N = len(train_data)
        random.seed(seed)
        train_subsample_ridx = random.sample(list(range(0, N)), int(N * prop_train))
        train_fit = train_data.iloc[train_subsample_ridx]
        fit_samples = [x for x in train_fit["s"]]
        evaluate_fit = train_data.loc[~train_data["s"].isin(fit_samples)]

        # Train RF
        training_set_known_labels = train_fit[known_col].values
        training_set_pcs = train_fit[pc_cols].values
        evaluation_set_pcs = evaluate_fit[pc_cols].values

        pop_clf = RandomForestClassifier(n_estimators=n_estimators, random_state=seed)
        pop_clf.fit(training_set_pcs, training_set_known_labels)
        print(
            "Random forest feature importances are as follows: {}".format(
                pop_clf.feature_importances_
            )
        )

        # Evaluate RF
        predictions = pop_clf.predict(evaluation_set_pcs)
        error_rate = 1 - sum(evaluate_fit[known_col] == predictions) / float(
            len(predictions)
        )
        print("Estimated error rate for RF model is {}".format(error_rate))
    else:
        pop_clf = fit

    # Classify data
    pop_pc_pd[output_col] = pop_clf.predict(pop_pc_pd[pc_cols].values)
    probs = pop_clf.predict_proba(pop_pc_pd[pc_cols].values)
    probs = pd.DataFrame(probs, columns=[f"prob_{p}" for p in pop_clf.classes_])
    pop_pc_pd = pd.concat([pop_pc_pd.reset_index(drop=True), probs.reset_index(drop=True)], axis=1)
    # pop_pc_pd = pd.concat([pop_pc_pd, probs], axis=1)
    probs["max"] = probs.max(axis=1)
    pop_pc_pd.loc[probs["max"] < min_prob, output_col] = missing_label
    pop_pc_pd = pop_pc_pd.drop(pc_cols, axis="columns")

    logger.info(
        "Found the following sample count after population assignment: %s",
        ", ".join(
            f"{pop}: {count}" for pop, count in Counter(pop_pc_pd[output_col]).items()
        ),
    )

    if hail_input:
        pops_ht = hl.Table.from_pandas(pop_pc_pd, key=list(pop_pca_scores.key))
        pops_ht = pops_ht.annotate_globals(
            assign_pops_from_pc_params=hl.struct(min_assignment_prob=min_prob)
        )

        if not fit:
            pops_ht = pops_ht.annotate_globals(
                assign_pops_from_pc_params=pops_ht.assign_pops_from_pc_params.annotate(
                    error_rate=error_rate
                )
            )

            pops_ht = pops_ht.annotate(
                evaluation_sample=hl.literal(list(evaluate_fit.s)).contains(pops_ht.s),
                training_sample=hl.literal(list(train_fit.s)).contains(pops_ht.s),
            )
        return pops_ht, pop_clf
    else:
        return pop_pc_pd, pop_clf

In [8]:
# gnomAD overwrites the df, so make a copy first
scores_df_gnomad = scores_df.copy()
gnomad_infered, gnomad_mod = assign_population_pcs_bug_fixed(scores_df_gnomad, colnames, known_col='SuperPop')

INFO (__main__ 156): Found the following sample count after population assignment: EUR: 651, oth: 50, EAS: 715, AMR: 381, CSA: 662, AFR: 744, OCE: 27, MID: 126


Random forest feature importances are as follows: [0.19354746 0.17704245 0.16032011 0.12148441 0.1125968  0.05376226
 0.04073679 0.0272037  0.02257492 0.01713482 0.01024832 0.01889163
 0.00742205 0.01143318 0.00380711 0.00130609 0.00644453 0.00429563
 0.00423795 0.00550978]
Estimated error rate for RF model is 0.0016528925619834212


### Export files for plotting in ggplot

In [9]:
# write out files for plotting in R
# gnomAD DF still contains training samples, remove them
gnomad_infered_filt = gnomad_infered[gnomad_infered['s'].isin(list(spcancestry_infered['s'].values))]
gnomad_infered_filt = gnomad_infered_filt.drop(columns=['SuperPop'])

# truth labels for annotating
truth_labels = pd.read_table(f'{path}hgdp_1kg_unknown_labels.txt',
                            header=0, names=['s', 'true_pop'])

# add TRUE population labels
spcancestry_infered_annotated = pd.merge(spcancestry_infered, truth_labels, on='s', how='left')
gnomad_infered_annotated = pd.merge(gnomad_infered_filt, truth_labels, on='s', how='left')

# write out data to files
spcancestry_infered_annotated.to_csv('spcancestry_probs.txt', sep='\t', index=False)
gnomad_infered_annotated.to_csv('gnomad_probs.txt', sep='\t', index=False)


# 2. Impact of the number of PCs on classification

In [10]:
# see how varying the number of PCs used as features impacts classification accuracy

# truth labels for annotating
truth_labels = pd.read_table(f'{path}hgdp_1kg_unknown_labels.txt',
                            header=0, names=['s', 'true_pop'])

def vary_pcs(scores_features_df: pd.DataFrame = None,
            num_pcs: int = None):
    """
        For investigating how classification accuracy changes with number of PCs used as features
    """
    # subset scores df to only just the number of specified PCs
    pcs_colnames = [f'PC{i+1}' for i in range(num_pcs)]
    cols_subset = ['s', 'SuperPop'] + pcs_colnames
    scores_subset = scores_features_df[cols_subset]
    
    # infer ancestry using spcancestry
    infered = spcancestry.infer_ancestry(scores_subset, pcs_colnames)
    infered_subset = infered[['s', 'pop']]
    infered_subset = infered_subset.rename(columns={'pop': f'pcs_{num_pcs}'})
    
    return infered_subset


merged_df = pd.DataFrame()

for n_pcs in [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]:
    pcs_df = vary_pcs(scores_df, n_pcs)
    
    if merged_df.empty:
        merged_df = pcs_df
    else:
        merged_df = merged_df.merge(pcs_df, on='s', how='left')
        
# Add true population labels
merged_df_annotated = merged_df.merge(truth_labels, on='s', how='left')

Estimated error rate for the meta model is 0.0380165289256198


/usr/local/lib/python3.9/site-packages/spcancestry/spcancestry.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_data.loc[:, output_col] = clf.predict(unknown_data[pc_cols].values)


Estimated error rate for the meta model is 0.0016528925619834212


/usr/local/lib/python3.9/site-packages/spcancestry/spcancestry.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_data.loc[:, output_col] = clf.predict(unknown_data[pc_cols].values)


Estimated error rate for the meta model is 0.0016528925619834212


...

/usr/local/lib/python3.9/site-packages/spcancestry/spcancestry.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_data.loc[:, output_col] = clf.predict(unknown_data[pc_cols].values)


Estimated error rate for the meta model is 0.0016528925619834212


/usr/local/lib/python3.9/site-packages/spcancestry/spcancestry.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_data.loc[:, output_col] = clf.predict(unknown_data[pc_cols].values)


Estimated error rate for the meta model is 0.0016528925619834212


/usr/local/lib/python3.9/site-packages/spcancestry/spcancestry.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_data.loc[:, output_col] = clf.predict(unknown_data[pc_cols].values)


In [11]:
pcs_cols = [f'pcs_{i}' for i in range(2, 22, 2)]

classification_counts = {'classification': ['Match', 'Mismatch', 'Unclassified']}

for npcs in pcs_cols:
    pcs_df = merged_df_annotated[[npcs, 'true_pop']]
    matches = (pcs_df[npcs] == pcs_df['true_pop']).sum()
    mismatches = ((pcs_df[npcs] != pcs_df['true_pop']) & ((pcs_df[npcs] != 'oth'))).sum()
    unclassified = (pcs_df[npcs] == 'oth').sum()
    assert (matches+mismatches+unclassified) == merged_df_annotated.shape[0]
    
    classification_counts[npcs] = [matches, mismatches, unclassified]
    
df_counts = pd.DataFrame(classification_counts)
df_counts

,classification,pcs_2,pcs_4,pcs_6,pcs_8,pcs_10,pcs_12,pcs_14,pcs_16,pcs_18,pcs_20
0,Match,299,334,333,333,333,333,333,333,333,333
1,Mismatch,6,0,0,0,0,0,0,0,0,0
2,Unclassified,30,1,2,2,2,2,2,2,2,2
